Ref Notebook: https://www.kaggle.com/jeffd23/visualizing-word-vectors-with-t-sne

Data: https://www.kaggle.com/c/quora-question-pairs/data

About t-SNE: https://www.datacamp.com/community/tutorials/introduction-t-sne

In [1]:
# Core Libraries and API's
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
import tweepy
import re
import nltk
from gensim.models import word2vec
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import pandas as pd
pd.options.mode.chained_assignment = None 

#Plot helpers
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import pylab
#Enable matplotlib to be interactive (zoom etc)
%matplotlib notebook

In [2]:
SW = SummaryWriter()

In [3]:
data = pd.read_csv('train.csv').sample(50000, random_state=23)

In [4]:
#nltk.download() if LookupError arises
STOP_WORDS = nltk.corpus.stopwords.words()

In [5]:
def clean_sentence(val):
    '''remove chars that are not letters or numbers, downcase, then remove stop words'''
    regex = re.compile('([^\s\w]|_)+')
    sentence = regex.sub('', val).lower()
    sentence = sentence.split(" ")
    
    for word in list(sentence):
        if word in STOP_WORDS:
            sentence.remove(word)  
            
    sentence = " ".join(sentence)
    return sentence

In [6]:
def clean_dataframe(data):
    '''drop nans, then apply 'clean_sentence' function to question1 and 2'''
    data = data.dropna(how="any")
    
    for col in ['question1', 'question2']:
        data[col] = data[col].apply(clean_sentence)
    
    return data

In [7]:
data = clean_dataframe(data)
data.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
237921,237921,9732,79801,sex necessary relationship,sex important good relationship,1
181001,181001,277377,277378,inspiring start stories,inspirational stories ever,0
294691,294691,150129,93109,best way digital marketing,best unique ways digital marketing,1
104145,104145,171986,171987,best way grow facebook fan page,get followers facebook page,1
357893,357893,487310,487311,suppose host sends two tcp segments back back ...,suppose host sends two tcp segments back back ...,1


In [8]:
def build_corpus(data):
    "Creates a list of lists containing words from each sentence"
    all_words = []
    corpus = []
    for col in ['question1', 'question2']:
        for sentence in data[col].iteritems():
#             print(sentence[0])
            word_list = sentence[1].split(" ")
            [all_words.append(x) for x in word_list]
            corpus.append(word_list)
            
    return corpus, all_words

In [9]:
corpus, vocab = build_corpus(data)        
# corpus[0:5]

In [10]:
model1 = word2vec.Word2Vec(corpus, size=100, window=20, min_count=200, workers=4)

In [11]:
# vecs, meta = [], []
vocab = list(model1.wv.vocab.keys())
vocab.remove('')
vecs = [model1.wv[x] for x in vocab]
meta = [x for x in vocab]
vecs = np.array(vecs)

In [12]:
SW.add_embedding(vecs, meta)

In [14]:
model1.most_similar("interview")

/home/rebaca/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('tips', 0.9015036821365356),
 ('process', 0.8848215341567993),
 ('job', 0.848846435546875),
 ('making', 0.7807451486587524),
 ('asked', 0.7799514532089233),
 ('answers', 0.6805151104927063),
 ('questions', 0.6780686378479004),
 ('answer', 0.6764006614685059),
 ('engineer', 0.6633208990097046),
 ('easily', 0.6507527828216553)]

In [16]:
model1.most_similar("trump")

/home/rebaca/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('president', 0.9017347097396851),
 ('election', 0.890000581741333),
 ('win', 0.8626320362091064),
 ('presidential', 0.8578933477401733),
 ('hillary', 0.849371075630188),
 ('america', 0.8198499083518982),
 ('donald', 0.8134077191352844),
 ('clinton', 0.7376631498336792),
 ('us', 0.7315635085105896),
 ('pakistan', 0.7063349485397339)]

In [ ]:
def tsne_plot_3D(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    z = []
    for value,idx in zip(new_values,range(len(new_values))):
        x.append(value[0])
        y.append(value[1])
        z.append(idx)
        
    plt.figure(figsize=(20, 20)) 
    ax = plt.axes(projection="3d")
    
    for i in range(len(x)):
        ax.scatter3D(z[i],x[i],y[i])
        ax.text(z[i],x[i],y[i], '%s' % str(labels[i]),size=10, zorder=1,  color='k')
        '''pylab.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(2, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')'''
    plt.show()

In [48]:
vocab

['sex',
 'necessary',
 'relationship',
 'inspiring',
 'start',
 'stories',
 'best',
 'way',
 'digital',
 'marketing',
 'best',
 'way',
 'grow',
 'facebook',
 'fan',
 'page',
 'suppose',
 'host',
 'sends',
 'two',
 'tcp',
 'segments',
 'back',
 'back',
 'host',
 'b',
 'tcp',
 'connection',
 'first',
 'segment',
 'sequence',
 'number',
 '140',
 'second',
 'sequence',
 'number',
 '170',
 'sahara',
 'average',
 'temperatures',
 'compare',
 'ones',
 'dashte',
 'loot',
 'possible',
 'disable',
 'facetimes',
 'call',
 'button',
 'agencies',
 'fulfil',
 'sex',
 'fantasies',
 'usa',
 'signs',
 'mentally',
 'weak',
 'different',
 'quora',
 'find',
 'list',
 'gmail',
 'addresses',
 'best',
 'resources',
 'learning',
 'sap',
 'hana',
 'best',
 'business',
 'opportunities',
 'india',
 'coding',
 'language',
 'learn',
 'first',
 'making',
 'games',
 'interesting',
 'products',
 'innovations',
 'brands',
 'coming',
 '2016',
 'question',
 'quora',
 'marked',
 'needing',
 'improvement',
 'make',
 'invi

In [10]:
model2 = word2vec.Word2Vec(corpus, size=20, window=20, min_count=200, workers=4)

In [11]:
vocab2 = list(model2.wv.vocab.keys())

In [12]:
vocab2.remove('')
vecs2 = [model2.wv[x] for x in vocab2]
meta2 = [x for x in vocab2]
vecs2 = np.array(vecs2)

In [13]:
SW.add_embedding(vecs2,meta2)

In [14]:
model2.wv["facebook"], model2.most_similar("facebook")

/home/rebaca/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


(array([ 0.07348374, -0.22552891,  1.2924196 ,  0.480665  , -0.95524997,
         1.2982266 ,  0.024901  ,  0.1764604 , -1.7280147 ,  0.3276524 ,
         0.21904837, -0.5483732 ,  0.53877825,  0.23125811, -1.3751838 ,
         0.5590445 , -0.11835239,  2.1472368 ,  1.1832741 ,  0.95622927],
       dtype=float32),
 [('profile', 0.9193522930145264),
  ('snapchat', 0.9117575883865356),
  ('instagram', 0.9108803868293762),
  ('delete', 0.8916009664535522),
  ('password', 0.8672595024108887),
  ('hack', 0.853140115737915),
  ('post', 0.8488222360610962),
  ('gmail', 0.8221774101257324),
  ('account', 0.8205013275146484),
  ('add', 0.8160465955734253)])

In [15]:
model2.wv["digital"], model2.most_similar("digital")

/home/rebaca/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


(array([-1.8820955 ,  1.5265124 , -1.3153312 , -0.15899405, -0.6577603 ,
         0.6795258 ,  0.6316236 , -1.4385037 , -0.8269056 , -0.610119  ,
        -0.3133054 ,  0.22227322,  0.16643932, -0.18023707, -0.19220729,
        -0.97044367,  2.3043964 , -0.03273883,  1.1947234 ,  1.2646352 ],
       dtype=float32),
 [('marketing', 0.980467677116394),
  ('course', 0.8774718046188354),
  ('websites', 0.7946375012397766),
  ('online', 0.783369243144989),
  ('free', 0.7531862258911133),
  ('sites', 0.752911388874054),
  ('available', 0.7300123572349548),
  ('easy', 0.7229315042495728),
  ('management', 0.7206048965454102),
  ('services', 0.7140038013458252)])

In [16]:
model2.wv["2016"], model2.most_similar("2016")

/home/rebaca/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


(array([-0.50815696,  1.9576399 ,  0.9397178 ,  0.26748452, -0.2526565 ,
        -0.218128  ,  0.07722626, -1.1996834 ,  0.9162859 , -0.6670083 ,
         0.5768697 ,  0.6844547 , -0.7781226 , -0.6521553 ,  0.77314764,
        -1.1394486 ,  0.8300363 ,  0.19884574,  0.85373217, -0.7682338 ],
       dtype=float32),
 [('2017', 0.7500601410865784),
  ('election', 0.746268630027771),
  ('win', 0.7291318774223328),
  ('presidential', 0.7171372771263123),
  ('next', 0.6955565810203552),
  ('score', 0.6698049306869507),
  ('preparation', 0.6595323085784912),
  ('gate', 0.6334092617034912),
  ('president', 0.6287544965744019),
  ('review', 0.6208642721176147)])

In [17]:
model2.similarity('2016', 'instagram'), model2.similarity('instagram', 'facebook')

/home/rebaca/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


(-0.14908484, 0.9108804)

In [64]:
model2.similarity('digital', 'marketing'), model2.similarity('marketing', 'hack')

/home/rebaca/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


(0.96631044, -0.021678349)